In [1]:
%load_ext autoreload
%autoreload 2
from preprocess import Preprocessor

log_train_path = "data/train/log_train.csv"
enroll_train_path = "data/train/enrollment_train.csv"
truth_train_path = "data/train/truth_train.csv"

p = Preprocessor(log_train_path, enroll_train_path, truth_train_path)

load course date info over!
('number of courses:', 39)
load course object info over!
('number of courses:', 39)
('number of moduels:', 26750)
load course object info over!
('size of log data:', 4677908)
('number of enrollment_ids:', 72395)
load enrollment info over!
('number of courses:', 39)
('number of enrollments:', 72395)
('number of uers: ', 53870)
load truth info over!
('number of labels:', 72395)


## Load features

In [79]:
df = p.get_features_all()

event_count features extracted! 1.532251 seconds taken
('Shape of the event_count dataframe: ', (72395, 8))
weekly_event_count features extracted! 1.896394 seconds taken
('Shape of the weekly_event_count dataframe: ', (72395, 7))
weekly_session_count features extracted! 81.006355 seconds taken
('Shape of the weekly_session_count dataframe: ', (72395, 6))
All features extracted! 84.678602 seconds taken
('Shape of the whole features dataframe: ', (72395, 20))


In [80]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_one_event,week_two_event,...,week_four_event,week_five_event,week_six_event,week_one_session,week_two_session,week_three_session,week_four_session,week_five_session,week_six_session,label
0,1,107,0,25,66,80,29,0,13,49,...,58,48,0,1,5,8,3,3,0,0
1,4,61,0,15,8,6,4,0,31,35,...,0,0,0,5,7,6,0,0,0,0
2,5,226,34,30,87,152,86,0,142,249,...,170,46,0,6,25,2,19,5,0,0
3,7,194,33,19,59,84,69,0,132,104,...,58,0,0,5,4,7,4,0,0,1
4,13,198,4,15,25,116,69,0,0,43,...,173,211,0,0,4,0,13,12,0,0


In [81]:
df.shape

(72395, 21)

## Deal with outliers

In [82]:
## Using the formula to calculate the outlier
## Using Quantile(0.75)+1.5IQR as upper cut-off line (As Boxplot)
## Using Quantile(0.25)-1.5IQR as lower cut-off line (As Boxplot)
## Transform the outliers to the nearest cut-off line
num_feature = list(df.columns[1:-1])
for item in num_feature:
    Q3 = df[item].quantile(0.75)
    Q1 = df[item].quantile(0.25)
    IQR = Q3-Q1
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    for i in range (0,72395):
        if df[item][i] < lower:
            df[item][i] = lower
        if df[item][i] > upper:
            df[item][i] = upper
df.shape

(72395, 21)

## Deal with low variance feature

In [83]:
## Dropping variables where 95% or more of the values are the same
## Find the ratio of the second most frequent value to the most frequent value for each predictor 
## Remove variables where this ratio was less than 0.05
num_feature = list(df.columns[1:-1])
for value in num_feature:
    freq_set = df[value].value_counts()
    max_1 = freq_set.max()
    max_2 = 0
    for num in freq_set:
        if num == max_1:
            continue
        max_2 = num
        break
    ratio = max_2/float(max_1)
    print value,":",ratio
    if ratio < 0.05:
        df = df.drop(value,1)      

access_count : 0.515257342417
discussion_count : 0.251805468587
navigate_count : 0.872527472527
page_close_count : 0.501494733937
problem_count : 0.268989795277
video_count : 0.432727544686
wiki_count : 0.197316790613
week_one_event : 0.316132234687
week_two_event : 0.264479741618
week_three_event : 0.277818275374
week_four_event : 0.27702577032
week_five_event : 0.0
week_six_event : 0.0
week_one_session : 0.0
week_two_session : 0.346970703585
week_three_session : 0.276481363492
week_four_session : 0.26095212261
week_five_session : 0.0
week_six_session : 0.0


In [84]:
df.shape

(72395, 16)

## Data transformation

In [85]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_one_event,week_two_event,week_three_event,week_four_event,week_two_session,week_three_session,week_four_session,label
0,1,61,0,22,25,12,15,0,7,32,17,15,2,2,2,0
1,4,61,0,15,8,6,4,0,7,32,17,0,2,2,0,0
2,5,61,5,22,25,12,15,0,7,32,8,15,2,2,2,0
3,7,61,5,19,25,12,15,0,7,32,17,15,2,2,2,1
4,13,61,4,15,25,12,15,0,0,32,0,15,2,0,2,0


- Normalization Data

In [86]:
num_feature = list(df.columns[1:-1])
for item in num_feature:
    df[item]=(df[item]-df[item].min())/(df[item].max()-df[item].min())

In [87]:
df.head()

,enrollment_id,access_count,discussion_count,navigate_count,page_close_count,problem_count,video_count,wiki_count,week_one_event,week_two_event,week_three_event,week_four_event,week_two_session,week_three_session,week_four_session,label
0,1,1.0,0.0,1.000000,1.00,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0
1,4,1.0,0.0,0.681818,0.32,0.5,0.266667,0.0,1.0,1.0,1.000000,0.0,1.0,1.0,0.0,0
2,5,1.0,1.0,1.000000,1.00,1.0,1.000000,0.0,1.0,1.0,0.470588,1.0,1.0,1.0,1.0,0
3,7,1.0,1.0,0.863636,1.00,1.0,1.000000,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1
4,13,1.0,0.8,0.681818,1.00,1.0,1.000000,0.0,0.0,1.0,0.000000,1.0,1.0,0.0,1.0,0
